# Project 1 - EEG

In [1]:
import dlc_bci as bci

import torch
from torch.autograd import Variable
from torch.nn import functional as F
from torch import Tensor

In [2]:
train_input , train_target = bci.load(root = './data_bci', one_khz=True)

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])


In [3]:
test_input, test_target = bci.load(root = './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
# Reshaping to (batch, seq, feature)
train_input = train_input.permute(0,2,1)
test_input = test_input.permute(0,2,1)

In [5]:
train_input.shape, test_input.shape

(torch.Size([316, 500, 28]), torch.Size([100, 500, 28]))

## Defining the model

In [6]:
seq = train_input.shape[1] # 500
input_size = train_input.shape[2] #28
hidden_size = 32
num_lstm_layers = 5
n_hidden_fc = 128

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # LSTM input size is: (batch, seq, feature)
        # LSTM output size is: (batch, seq, hidden_size)
        self.lstm = torch.nn.LSTM(input_size=input_size,
                                  hidden_size=hidden_size,
                                  num_layers=num_lstm_layers,
                                  batch_first=True)
        self.fc1 = torch.nn.Linear(seq*hidden_size, n_hidden_fc)
        self.fc2 = torch.nn.Linear(n_hidden_fc, 2)
        self.sm = torch.nn.Softmax(1)

    def forward(self, x):
        x = self.lstm(x)[0]
        x = x.contiguous().view(x.shape[0], -1) # flatten to (batch, seq*hidden_size)
        x = F.relu(self.fc1(x))
        x = self.sm(self.fc2(x))
        return x
    
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (Variable(torch.zeros(4, num_lstm_layers, hidden_size)),
                Variable(torch.zeros(4, num_lstm_layers, hidden_size)))

In [7]:
def train_model(model, train_input, train_target, mini_batch_size, n_epochs, verbose=0):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    eta = 1
    
    for e in range(n_epochs):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            model.zero_grad()
            model.hidden = model.init_hidden()
            
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.data[0]
            loss.backward()
            optimizer.step()
        if verbose:
            print(e, sum_loss)

In [8]:
def compute_nb_errors(model, input, target, mini_batch_size):

    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = output.data.max(1)[1]
        true_classes = target.data.narrow(0, b, mini_batch_size)
        nb_errors += (predicted_classes == true_classes).sum()

    return nb_errors

In [9]:
train_input_var, train_target_var = Variable(train_input), Variable(train_target.long())
test_input_var, test_target_var = Variable(test_input), Variable(test_target.long())

In [18]:
mini_batch_size = 4
epochs = 2

model = Net()
model.train()
train_model(model, train_input_var, train_target_var, mini_batch_size, epochs, verbose=True)
model.eval()
nb_test_errors = compute_nb_errors(model, test_input_var, test_target_var, mini_batch_size)
print('Test error: {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input_var.size(0),
                                                   nb_test_errors, test_input_var.size(0)))

nb_train_errors = compute_nb_errors(model, train_input_var, train_target_var, mini_batch_size)
print('Train error: {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input_var.size(0),
                                                   nb_train_errors, train_input_var.size(0)))

0 55.25463652610779
1 54.89609384536743
Test error: 49.00% 49/100
Train error: 50.32% 159/316


In [12]:
train_input[[4,5,6,7,8,21,32,53]].shape

torch.Size([8, 500, 28])

In [14]:
out = model.forward(train_input_var[[4,5,6,7,8,21,32,53]])
out

Variable containing:
 0.5103  0.4897
 0.5103  0.4897
 0.5104  0.4896
 0.5104  0.4896
 0.5103  0.4897
 0.5103  0.4897
 0.5103  0.4897
 0.5104  0.4896
[torch.FloatTensor of size 8x2]

In [15]:
train_target_var[[4,5,6,7,8,21,32,53]]

Variable containing:
 0
 1
 0
 1
 0
 0
 1
 1
[torch.LongTensor of size 8]